In [8]:
import pandas as pd
import datetime
import numpy as np
from tqdm import tqdm_notebook
from datetime import datetime,timedelta
import os
from timeit import default_timer as timer
from scipy import stats
os.chdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list=os.listdir('/Users/hudsonyeo/Desktop/Python/leo/data/day/TA')
file_list.sort()

In [12]:
file_list

['.DS_Store',
 '2019.01.02.csv',
 '2019.01.03.csv',
 '2019.01.04.csv',
 '2019.01.07.csv',
 '2019.01.08.csv',
 '2019.01.09.csv',
 '2019.01.10.csv',
 '2019.01.11.csv',
 '2019.01.14.csv',
 '2019.01.15.csv',
 '2019.01.16.csv',
 '2019.01.17.csv',
 '2019.01.18.csv',
 '2019.01.21.csv',
 '2019.01.22.csv',
 '2019.01.23.csv',
 '2019.01.24.csv',
 '2019.01.25.csv',
 '2019.01.28.csv',
 '2019.01.29.csv',
 '2019.01.30.csv',
 '2019.01.31.csv',
 '2019.02.01.csv',
 '2019.02.11.csv',
 '2019.02.12.csv',
 '2019.02.13.csv',
 '2019.02.14.csv',
 '2019.02.15.csv',
 '2019.02.18.csv',
 '2019.02.19.csv',
 '2019.02.20.csv',
 '2019.02.21.csv',
 '2019.02.22.csv',
 '2019.02.25.csv',
 '2019.02.26.csv',
 '2019.02.27.csv',
 '2019.02.28.csv',
 '2019.03.01.csv',
 '2019.03.04.csv',
 '2019.03.05.csv',
 '2019.03.06.csv',
 '2019.03.07.csv',
 '2019.03.08.csv',
 '2019.03.11.csv',
 '2019.03.12.csv',
 '2019.03.13.csv',
 '2019.03.14.csv',
 '2019.03.15.csv',
 '2019.03.18.csv',
 '2019.03.19.csv',
 '2019.03.20.csv',
 '2019.03.21.csv'

In [9]:
def calc_smart_price(dataset):
    data=dataset[:]
    data['smart_price']=data.loc[:,'BidPrice1']*data.loc[:,'AskVol1']+data.loc[:,'AskPrice1']*data.loc[:,'BidVol1']
    data.at[:,'smart_price']=data.loc[:,'smart_price']/(data.loc[:,['BidVol1','AskVol1']].sum(axis=1))  
    return data

def calc_future_price(dataset,time_ahead=30):
    data=dataset[:]
    future_price=[]
    length=len(data)
    for i in range(len(data)):
        current_time=data[i,44]+timedelta(seconds=time_ahead)
        #print(data[i,44])
        j=0
        #print(current_time)
        while((i+j)<length and current_time>data[(i+j),44]):
            j+=1
        #print(i,j,(data[(i+j-1),44]))
        if (i+j)<length:
            future_price.append(data[(i+j),51]) #51 is the index for smart price            
        else:
            future_price.append(np.nan)
    future_price=np.asarray(future_price)
    future_price=np.expand_dims(future_price,axis=1)
    return np.concatenate((data,future_price),axis=1)


def calc_edge(dataset):
    data=dataset.copy()
    temp=data[:,52]-data[:,51]
    temp=np.expand_dims(temp,axis=1)
    return np.concatenate((data,temp),axis=1)

def set_index(dataset):
    data=dataset[:]
    index=data[:,44]
    new_index=[]
    for j in range(len(index)):
        i=str(index[j]*1000)
        if len(i)==11:
            i='0'+i
        i=i[:-10]+':'+i[-10:]
        i=i[:-8]+':'+i[-8:]
        i=i[:-6]+':'+i[-6:]
        new_index.append(datetime.strptime(i,"%H:%M:%S:%f"))
    data[:,44]=new_index
    return data
def calc_sma(dataset,duration=1): #not in use, just for reference
    #5 seconds for 5000 rows
    data=dataset[:]
    sma_values=[]

    for i in data[:,44]:
        last_time=i-timedelta(minutes=duration)
        sma=data[(data[:,44]>=last_time) & (data[:,44]<i)]
        if len(sma)!=0:
            sma=sma[:,51].mean()
        else:
            sma=np.nan
        sma_values.append(sma)
    sma_values=np.asarray(sma_values)
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1)
def calc_sma_fast(dataset,duration=1): #faster way to calculate SMA, 0.05 seconds for 5000 rows
    data=dataset[:]
    sma_values=[] 
    smart_sum=np.cumsum(data[:,51])
    for i in range(len(data)):
        last_time=data[i,44]-timedelta(minutes=duration)
        j=220*duration#4x60=240
        while(i-j>0 and data[i-j,44]>last_time):
            j+=1
        if (i-j>=0):
            sma=(smart_sum[i]-smart_sum[i-j])/(j)
            sma_values.append(sma)
        else:
            sma=smart_sum[i]/(i+1)
            sma_values.append(sma)
    sma_values=np.asarray(sma_values)
    sma_values=sma_values-data[:,51]
    sma_values=np.expand_dims(sma_values,axis=1)
    return np.concatenate((data,sma_values),axis=1)        
def process(dataset):
    data=dataset[:]
    data=calc_smart_price(data).values
    data=set_index(data)
    data=calc_future_price(data)
    data=calc_edge(data)
    data=calc_sma_fast(data,duration=1)
    data=calc_sma_fast(data,duration=5)
    data=calc_sma_fast(data,duration=15)
    data=calc_sma_fast(data,duration=30)    
    return data
    
        
'''def calc_vwap(dataset,duration=1): #to be implement
    data=dataset[:]
    for i in data[:,44]:
        last_time=i-timedelta(minutes=duration)
        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]
        high=rolling[:,51].max()
        low=rolling[:,51].min()
        avg=(rolling[-1,51]+high+low)/3
def calc_rsi(dataset)        '''        

'def calc_vwap(dataset,duration=1):\n    data=dataset[:]\n    for i in data[:,44]:\n        last_time=i-timedelta(minutes=duration)\n        rolling=data[(data[:,44]>=last_time) & (data[:,44]<i)]\n        high=rolling[:,51].max()\n        low=rolling[:,51].min()\n        avg=(rolling[-1,51]+high+low)/3'

In [3]:
ma_dict={'-4':'1',
        '-3':'5',
        '-2':'15',
        '-1':'30'}

In [ ]:
file_list[1][-3:]
df_list=[]
name_list=[]
path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/'
for file in file_list: #read all files and add them to file_list
    if file[-3:]=='csv': #check if file is a CSV
        name_list.append(file)
        df_list.append(process(pd.read_csv(path+file)))
        print(file,'read')

In [13]:
len(df_list)

83

In [11]:
#
df_path='/Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/'

num_days=4
for j in range(num_days): #for each column of moving average, -1 column is 30 min MA, -2 is 15 min, -3 is 5 min, -4 is 1 minute
    final_df=pd.DataFrame()    
    print(j,'ma')    
    j=j+1
    j*=-1

    for i in range(len(df_list)):
        if i<4:
            continue
        x=np.concatenate((df_list[i-4][:,j],df_list[i-3][:,j],df_list[i-2][:,j],df_list[i-1][:,j],df_list[i][:,j])).astype(float)

        y=np.concatenate((df_list[i-4][:,-5],df_list[i-3][:,-5],df_list[i-2][:,-5],df_list[i-1][:,-5],df_list[i][:,-5])).astype(float)
        isnum=(~np.isnan(x)) & (~np.isnan(y))
        y=y[isnum]
        x=x[isnum]        
        result=stats.linregress(x,y)
        final_df=final_df.append({'date':name_list[i],
                     'ma_time_minutes':ma_dict.get(str(j)),
                     'slope':result[0],
                     'intercept':result[1],
                     'r_val':result[2],
                     'p_val':result[3],
                     'std_err':result[4],
                     'x_mean':np.mean(x),
                     'x_std':np.std(x)},ignore_index=True)
    temp=df_path+'result_5day_'+ma_dict.get(str(j))+'minute_ma.csv'
    final_df.to_csv(temp)
    print('done',temp)

0 ma
done /Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_5day_30minute_ma.csv
1 ma
done /Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_5day_15minute_ma.csv
2 ma
done /Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_5day_5minute_ma.csv
3 ma
done /Users/hudsonyeo/Desktop/Python/leo/data/day/TA/results/result_5day_1minute_ma.csv
